<!--Información del curso-->
<img align="left" style="padding-right:10px;" src="figuras/logo_ciencia_datos.png">

<center><h1 style="font-size:2em;color:#2467C0"> Pandas -Parte 5 </h1></center>

<center><h2 style="font-size:2em;color:#840700">  Manejo de datos faltantes  </h4></center>

<br>
<table>
<col width="550">
<col width="450">
<tr>
<td><img src="figuras/nan_presentacion.png" align="left" style="width:500px"/></td>
<td>

* **Wes McKinney**, empezó a desarrollar Pandas en el año 2008 mientras trabajaba en *AQR Capital* [https://www.aqr.com/] por la necesidad que tenía de una herramienta flexible de alto rendimiento para realizar análisis cuantitativos en datos financieros. 
* Antes de dejar AQR convenció a la administración de la empresa de distribuir esta biblioteca bajo licencia de código abierto.
* **Pandas** es un acrónimo de **PANel DAta analysiS**
   
    
<br>
</td>
</tr>
</table>

# Librerias

In [1]:
import numpy as np
import matplotlib.pyplot as plt

#  Introducción

La diferencia entre los datos encontrados en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez estan limpios y homogéneos.
En particular, muchos conjuntos de datos interesantes tendrán cierta cantidad de datos faltantes. Para complicar aún más las cosas, diferentes fuentes de datos pueden indicar datos faltantes de diferentes maneras.

En esta lección se discutirán algunas consideraciones generales para los datos faltantes, discutiremos cómo Pandas elige representarlos y se mostrarán algunas herramientas integradas de Pandas para manejar datos faltantes en Python.
Se hará referencia a los datos faltantes de manera general como valores * nulo *, * NaN * o * NA *.

# Datos faltantes en pandas

En el desarrollo de Pandas se eligió usar centinelas para los datos faltantes, y se eligió usar dos valores nulos de Python ya existentes: el valor especial de punto flotante **NaN** y el objeto Python **None**. Esta elección tiene algunos efectos secundarios, pero en la práctica termina siendo una buena elección en la mayoría de los casos de interés.

###  ``None``: Para datos faltantes de Python

El primer valor centinela utilizado por Pandas es ``None``, un objeto simple de Python que a menudo se usa para datos faltantes en el código de Python. Debido a que es un objeto Python, ``None`` no se puede usar en un arreglo número de de NumPy/Pandas.

### ``NaN``: Dato número faltante
 ``NaN`` es el acrónimo de ***Not a Number***. Es un valor especial de punto flotante reconocido por todos los sistemas que utilizan la representación estándar de punto flotante IEEE:

In [2]:
import numpy as np
import pandas as pd

valores = np.array([1, np.nan, 3, 4]) 
valores.dtype

dtype('float64')

NumPy eligió un tipo de punto flotante nativo para esta matriz: esto significa que, este arreglo admite operaciones como cualquier otro arreglo. **Debe tener en cuenta que `` NaN`` es  como un virus de datos: infecta cualquier otro objeto que toque**. Independientemente de la operación, el resultado de la aritmética con ``NaN`` será otro ``NaN``:

In [3]:
1 + np.nan

nan

In [4]:
0 *  np.nan

nan

Tenga en cuenta que esto significa que los **agregados** sobre los arreglos no generarán un error  pero no se tendrán resultados útiles:

In [5]:
valores.sum(), valores.min(), valores.max()

(nan, nan, nan)

NumPy proporciona algunos **agregados** especiales que ignorarán estos valores faltantes:

In [6]:
np.nansum(valores), np.nanmin(valores), np.nanmax(valores)

(8.0, 1.0, 4.0)

Importante: tenga en cuenta que ``NaN`` es específicamente un valor de punto flotante; no hay un valor equivalente de NaN para enteros, strings u otros tipos.

### NaN y None en Pandas

``NaN`` y ``None`` tienen su lugar, y Pandas está diseñado para manejarlos de una sola manera:

In [7]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

El enfoque de centinela  de Pandas funciona bastante bien en la práctica y rara vez causa problemas. 

## Operando con valores nulos

Como se ha visto, Pandas trata ``None`` y ``NaN`` de igual manera para indicar valores faltantes o nulos.
Para facilitar esta convención, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas.
Son:

- ``isnull ()``: genera una máscara booleana que indica valores faltantes
- ``notnull ()``: opuesto a ``isnull ()``
- ``dropna ()``: Devuelve una versión filtrada de los datos
- ``fillna ()``: Devuelve una copia de los datos con valores faltantes rellenados 

### Detectando valores nulos
Las estructuras de datos de Pandas tienen dos métodos útiles para detectar datos nulos: ``isnull ()`` y ``notnull ()``.
Cualquiera de los dos devolverá una máscara booleana sobre los datos. Por ejemplo:

In [8]:
data = pd.Series([1, np.nan, 'hello', None])

In [9]:
# Aplicar la función isnull a la serie data
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [10]:
# Aplicar la función notnull a la serie data
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

Se puede por tanto generarar máscaras booleanas para un ``Serie`` o un ``DataFrame``

In [11]:
#Generar una serie de elementos no nulos de la serie data
data[data.notnull()]

0        1
2    hello
dtype: object

In [12]:
#Generar una serie de elementos nulos  de la serie data
data[data.isnull()]

1     NaN
3    None
dtype: object

### Descartando valores nulos

Además de las máscaras utilizadas anteriormente, existen los siguientes métodos: ``dropna ()``
(que elimina los valores de NA) y ``fillna ()`` (que completa los valores de NaN). Para una ``Serie``,
El resultado es sencillo:

In [13]:
#Aplicar la funcion dropna a la serie data
data.dropna()

0        1
2    hello
dtype: object

Para un ``DataFrame``, hay más opciones. Considere el siguiente ``DataFrame``:

In [14]:
df = pd.DataFrame([[1,      np.nan, 2.],
                   [2,      3,      5.],
                   [np.nan, 4,      6.]])
df

,0,1,2
0,1.0,NaN,2.0
1,2.0,3.0,5.0
2,NaN,4.0,6.0


No se puede eliminar valores individuales de un ``DataFrame``; solo se puede eliminar filas completas o columnas completas.
Dependiendo del problema que se tenga, es posible que desee uno u otro, por lo que ``dropna ()`` ofrece una serie de opciones para un ``DataFrame``. 

Por defecto, ``dropna ()`` eliminará todas las filas en las que *cualquier* valor nulo (NaN) esté presente:

In [15]:
#dropna()
df.dropna()

,0,1,2
1,2.0,3.0,5.0


Alternativamente, puede eliminar los valores de NaN a lo largo de un eje; ``axis = 1`` elimina todas las columnas que contienen un valor nulo:

In [16]:
#dropna(axis=1)
df.dropna(axis=1)

,2
0,2.0
1,5.0
2,6.0


In [17]:
# Puede usar axis='columns' para decir explicitamente que solo se afetaràn las columnas
# dropna(axis='columns')
df.dropna(axis='columns')

,2
0,2.0
1,5.0
2,6.0


In [18]:
# Puede usar axis='rows' para decir explicitamente que solo se afetaràn las fila
# dropna(axis='rows')
df.dropna(axis='rows')

,0,1,2
1,2.0,3.0,5.0


Pero esto también puede eliminar datos importantes; es posible eliminar solo filas o columnas que cumplan con alguna condición. Esto se puede especificar a través de los parámetros ``how`` o ``thresh``, que permiten un control preciso del número de elementos nulos para eliminar.

El valor predeterminado es ``how='any'``, de modo que cualquier fila o columna (dependiendo de la palabra clave ``axis``) que contenga un valor nulo se eliminará. También se puede especificar ``how='all'``, que solo eliminará filas/columnas que tengan  son *todos* sus valores nulos:


In [19]:
# Agrega al DataFrame,df, una nueva columna llamada 3 y que todos sus los elementos sean nulos (np.nan)
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2.0,NaN
1,2.0,3.0,5.0,NaN
2,NaN,4.0,6.0,NaN


In [20]:
#Eliminar las columnas que tengan todos sus elementos nulos
df.dropna(axis=1, how='all')

,0,1,2
0,1.0,NaN,2.0
1,2.0,3.0,5.0
2,NaN,4.0,6.0


In [21]:
#Eliminar las columnas que contengan al menos un elemento nulo
df.dropna(axis=1, how='any')

,2
0,2.0
1,5.0
2,6.0


Para un control más cuidadoso, el parámetro ``thresh``  permite especificar un número mínimo de valores **no nulos** para las filas/columnas que se mantendrán:

In [22]:
df[4]= pd.DataFrame([ 1.0,2.0, 3.0])
df[5]= pd.DataFrame([ 1.0,2.0, np.nan])
df[6]= pd.DataFrame([ 1.0,np.nan, np.nan])
df[7]= pd.DataFrame([ np.nan,np.nan, np.nan])

df

,0,1,2,3,4,5,6,7
0,1.0,NaN,2.0,NaN,1.0,1.0,1.0,NaN
1,2.0,3.0,5.0,NaN,2.0,2.0,NaN,NaN
2,NaN,4.0,6.0,NaN,3.0,NaN,NaN,NaN


1. Para las columnas

In [23]:
#thresh = 1
df.dropna(axis=1, thresh=1)

,0,1,2,4,5,6
0,1.0,NaN,2.0,1.0,1.0,1.0
1,2.0,3.0,5.0,2.0,2.0,NaN
2,NaN,4.0,6.0,3.0,NaN,NaN


In [24]:
#thresh = 2
df.dropna(axis=1, thresh=2)

,0,1,2,4,5
0,1.0,NaN,2.0,1.0,1.0
1,2.0,3.0,5.0,2.0,2.0
2,NaN,4.0,6.0,3.0,NaN


In [25]:
#thresh = 3
df.dropna(axis=1, thresh=3)

,2,4
0,2.0,1.0
1,5.0,2.0
2,6.0,3.0


2. Para las filas

In [26]:
#thresh = 3
df.dropna(thresh=3)

,0,1,2,3,4,5,6,7
0,1.0,NaN,2.0,NaN,1.0,1.0,1.0,NaN
1,2.0,3.0,5.0,NaN,2.0,2.0,NaN,NaN
2,NaN,4.0,6.0,NaN,3.0,NaN,NaN,NaN


In [27]:
#thresh = 4
df.dropna(thresh=4) 

,0,1,2,3,4,5,6,7
0,1.0,NaN,2.0,NaN,1.0,1.0,1.0,NaN
1,2.0,3.0,5.0,NaN,2.0,2.0,NaN,NaN


Importante: Recuerde que ``thresh``  se refiere a valores **no nulos**

### Reemplazando  valores nulos

A veces, en lugar de descartar los valores NaN, convendría mejor reemplazarlos con un valor válido. *Este valor podría ser un número único como cero, o podría ser algún tipo de asignación o interpolación de los buenos valores.* Pandas proporciona el método ``fillna()``, que devuelve una copia de la matriz con valores nulos reemplazados.

Considere la siguiente ``Serie``:


In [28]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

Podemos llenar entradas de NA con un solo valor, como cero:

In [29]:
#Rellenar con 0
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [30]:
#Rellenar con -10
data.fillna(-10) 

a     1.0
b   -10.0
c     2.0
d   -10.0
e     3.0
dtype: float64

Se puede especificar un relleno hacia adelante (forward-fill) , para propagar el valor anterior hacia adelante:

In [31]:
# forward-fill
data.fillna(method='ffill') 

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

O se puede especificar un relleno para propagar los siguientes valores hacia atrás (back-fill ):

In [32]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

Para los *DataFrames*, las opciones son similares, pero también podemos especificar un eje,``axis``, a lo largo del cual tienen lugar el relleno:


In [33]:
df.columns=list('abcdefgh')
df

,a,b,c,d,e,f,g,h
0,1.0,NaN,2.0,NaN,1.0,1.0,1.0,NaN
1,2.0,3.0,5.0,NaN,2.0,2.0,NaN,NaN
2,NaN,4.0,6.0,NaN,3.0,NaN,NaN,NaN


In [34]:
#fillna(method='ffill', axis=1)
df.fillna(method='ffill', axis=1)

,a,b,c,d,e,f,g,h
0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0
1,2.0,3.0,5.0,5.0,2.0,2.0,2.0,2.0
2,NaN,4.0,6.0,6.0,3.0,3.0,3.0,3.0


Observe que si un valor anterior no está disponible durante un llenado hacia adelante, el valor NA permanece.

In [35]:
# fillna(method='ffill', axis=0)
df.fillna(method='ffill', axis=0)

,a,b,c,d,e,f,g,h
0,1.0,NaN,2.0,NaN,1.0,1.0,1.0,NaN
1,2.0,3.0,5.0,NaN,2.0,2.0,1.0,NaN
2,2.0,4.0,6.0,NaN,3.0,2.0,1.0,NaN


# Ejercicios

<div class="alert alert-success">
    
1. Para el siguiente diccionario, defina un DataFrame y realice las operaciones para obtener la salida que se muestra 

dict = {'First Score':[100, 90, np.nan, 95], 
        'Second Score': [30, 45, 56, np.nan], 
        'Third Score':[np.nan, 40, 80, 98]} 

    
<img align="left" width="350"  float= "none" align="middle" src="figuras/nan1.png">

    
</div> 

In [36]:
# Creación del DataFrame
dict = {'First Score':[100, 90, np.nan, 95], 'Second Score': [30, 45, 56, np.nan], 'Third Score':[np.nan, 40, 80, 98]}
df = pd.DataFrame(dict)
df

,First Score,Second Score,Third Score
0,100.0,30.0,NaN
1,90.0,45.0,40.0
2,NaN,56.0,80.0
3,95.0,NaN,98.0


In [37]:
df.fillna(0)

,First Score,Second Score,Third Score
0,100.0,30.0,0.0
1,90.0,45.0,40.0
2,0.0,56.0,80.0
3,95.0,0.0,98.0


<div class="alert alert-success">
    
2. Para el siguiente diccionario, defina un DataFrame y realice las operaciones para obtener la salida que se muestra 

dict = {'First Score':[100, 90, np.nan, 95], 
        'Second Score': [30, 45, 56, np.nan], 
        'Third Score':[np.nan, 40, 80, 98]} 

    
<img align="left" width="350"  float= "none" align="middle" src="figuras/nan2.png">
    
</div> 

In [38]:
# Creación del DataFrame
dict = {'First Score':[100, 90, np.nan, 95], 'Second Score': [30, 45, 56, np.nan], 'Third Score':[np.nan, 40, 80, 98]}
df = pd.DataFrame(dict)
df

,First Score,Second Score,Third Score
0,100.0,30.0,NaN
1,90.0,45.0,40.0
2,NaN,56.0,80.0
3,95.0,NaN,98.0


In [39]:
df.fillna(method='ffill')

,First Score,Second Score,Third Score
0,100.0,30.0,NaN
1,90.0,45.0,40.0
2,90.0,56.0,80.0
3,95.0,56.0,98.0


<div class="alert alert-success">
    
3. Para el siguiente diccionario, defina un DataFrame y realice las operaciones para obtener la salida que se muestra 

dict = {'First Score':[100, 90, np.nan, 95], 
        'Second Score': [30, np.nan, 45, 56], 
        'Third Score':[52, 40, 80, 98], 
        'Fourth Score':[np.nan, np.nan, np.nan, 65]} 
    
    
    
<img align="left" width="450"  float= "none" align="middle" src="figuras/nan3.png">

</div> 

In [40]:
# Creación del DataFrame
dict = {'First Score':[100, 90, np.nan, 95], 'Second Score': [30, np.nan, 45, 56], 'Third Score':[52, 40, 80, 98], 'Fourth Score':[np.nan, np.nan, np.nan, 65]}
df = pd.DataFrame(dict)
df

,First Score,Second Score,Third Score,Fourth Score
0,100.0,30.0,52,NaN
1,90.0,NaN,40,NaN
2,NaN,45.0,80,NaN
3,95.0,56.0,98,65.0


In [41]:
df.dropna()

,First Score,Second Score,Third Score,Fourth Score
3,95.0,56.0,98,65.0


<div class="alert alert-success">
    
4. Para el siguiente diccionario, defina un DataFrame y realice las operaciones para obtener la salida que se muestra 

dict = {'First Score':[100, np.nan, np.nan, 95], 
        'Second Score': [30, np.nan, 45, 56], 
        'Third Score':[52, np.nan, 80, 98], 
        'Fourth Score':[np.nan, np.nan, np.nan, 65]} 

    
    
<img align="left" width="350"  float= "none" align="middle" src="figuras/nan4.png">

</div> 

In [42]:
# Creación del DataFrame
dict = {'First Score':[100, np.nan, np.nan, 95], 'Second Score': [30, np.nan, 45, 56], 'Third Score':[52, np.nan, 80, 98], 'Fourth Score':[np.nan, np.nan, np.nan, 65]}
df = pd.DataFrame(dict)
df

,First Score,Second Score,Third Score,Fourth Score
0,100.0,30.0,52.0,NaN
1,NaN,NaN,NaN,NaN
2,NaN,45.0,80.0,NaN
3,95.0,56.0,98.0,65.0


In [43]:
df.dropna(thresh=1)

,First Score,Second Score,Third Score,Fourth Score
0,100.0,30.0,52.0,NaN
2,NaN,45.0,80.0,NaN
3,95.0,56.0,98.0,65.0


<div class="alert alert-success">
    
5. Para el siguiente diccionario, defina un DataFrame y realice las operaciones para obtener la salida que se muestra 

dict = {'First Score':[100, np.nan, np.nan, 95], 
        'Second Score': [30, np.nan, 45, 56], 
        'Third Score':[52, np.nan, 80, 98], 
        'Fourth Score':[60, 67, 68, 65]} 

    
<img align="left" width="150"  float= "none" align="middle" src="figuras/nan5.png">
    
</div> 

In [44]:
# Creación del DataFrame
dict = {'First Score':[100, np.nan, np.nan, 95], 'Second Score': [30, np.nan, 45, 56], 'Third Score':[52, np.nan, 80, 98], 'Fourth Score':[60, 67, 68, 65]}
df = pd.DataFrame(dict)
df

,First Score,Second Score,Third Score,Fourth Score
0,100.0,30.0,52.0,60
1,NaN,NaN,NaN,67
2,NaN,45.0,80.0,68
3,95.0,56.0,98.0,65


In [45]:
df.dropna(axis=1)

,Fourth Score
0,60
1,67
2,68
3,65


<div class="alert alert-success">
    
6. Utilizando el DataFrame del archivo "datos/titanic.csv", encontrar las columnas que tienen elementos faltantes
    
</div> 

In [46]:
df = pd.read_csv('datos/titanic.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [47]:
datos_faltantes = df.isnull().sum()
datos_faltantes

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [48]:
print('Las columnas que tienen datos faltantes son:')
for key, value in zip(datos_faltantes.index, datos_faltantes):
    if value != 0:
        print(key)

Las columnas que tienen datos faltantes son:
Age
Cabin
Embarked


<div class="alert alert-success">
    
7. Utilizando el DataFrame del archivo "datos/titanic.csv", encontrar el número de elementos faltantes en cada una de las columnas.
    
</div> 

Utilizando la serie creada en el ejercicio anterior:

In [49]:
print('Los datos falantes por columna son:')
for key, value in zip(datos_faltantes.index, datos_faltantes):
    if value != 0:
        print('A la columna', key, 'le faltan', value, 'datos.')

Los datos falantes por columna son:
A la columna Age le faltan 177 datos.
A la columna Cabin le faltan 687 datos.
A la columna Embarked le faltan 2 datos.


<div class="alert alert-success">
    
8. Utilizando el DataFrame del archivo "datos/titanic.csv", cambiar los elementos NaN de la columna "Age" por -100.
    
</div> 

In [50]:
df['Age'] = df['Age'].fillna(-100)
print('Datos NaN en la columna Age:' ,df['Age'].isnull().sum())
df

Datos NaN en la columna Age: 0


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,-100.0,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


<div class="alert alert-success">
    
9. Utilizando el DataFrame del archivo "datos/titanic.csv", cambiar los elementos NaN de la columna "Cabin" por "no record".
    
</div> 

In [51]:
df['Cabin'] = df['Cabin'].fillna('NO RECORD')
print('Datos NaN en la columna Cabin:' ,df['Cabin'].isnull().sum())
df

Datos NaN en la columna Cabin: 0


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NO RECORD,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NO RECORD,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NO RECORD,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NO RECORD,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,-100.0,1,2,W./C. 6607,23.4500,NO RECORD,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


<div class="alert alert-success">
    
10. Utilizando el DataFrame del archivo "datos/titanic.csv", generar un nuevo dataframe considerando solo filas que no tengan datos faltantes (es decir filas con datos). ¿Cuántas filas quedaron?  
    
</div> 

In [52]:
df = pd.read_csv('datos/titanic.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [53]:
df = df.dropna()
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [54]:
print('Quedaron', len(df), 'filas')

Quedaron 183 filas


# Conclusión

<div class="alert alert-success">

11. Escribir alguna breve conclusión de lo aprendido en la actividad 4  y 5

</div>

En estas notebooks, creo yo, ya empezamos con el análisis de datos. Con las actividades anteriores aprendimos algunas cosas generales: lectura y creación de DataFrames; sin embargo, con estos cuadernos ya empezamos a estructurar el dataframe, juntar varios de estos para obtener la mayor cantidad de información posible y, por otro lado, eliminar o remplazar algunos valores en los que la infromación no existe, esta es una parte importante de todo el proceso de análisis de datos, es la base del proyecto que llevaremos a cabo.